# Long Short-Term Memory (LSTM)

* Import
* Custom Dataset
* Dataset DataLoader
* ** LSTM Model **
* Loss Function & Optimizer
* Training Model
* Testing 

**Reference**

* LSTM Pytorch [Doc](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

## Import

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.optim as optim

## Custom Dataset

In [5]:
class StockDataset(Dataset):
    def __init__(self, seq_length, file_path, train=True, transform=None):
        
        # read file
        df = pd.read_excel(file_path)
        df_len = len(df) - 1

        df_len -= ( df_len % seq_length )
        total_size = df_len // seq_length
        train_size = int(total_size * 0.7)
        
        f = df.iloc[:(train_size*seq_length), 3:7] # data
        l = df.iloc[1:(train_size*seq_length)+1, 7:] #label
        data_size = train_size
        
        if not train:
            f = df.iloc[(train_size*seq_length):df_len, 3:7] # data
            l = df.iloc[(train_size*seq_length)+1:df_len+1, 7:] #label
            data_size = total_size - train_size
        
        self.features = np.array(f).reshape(data_size, seq_length, 4)
        self.labels = np.array(l).reshape(data_size, seq_length, 1)
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):

        features = self.features[index]
        labels = self.labels[index]
        
        if self.transform:
            features = self.transform(features)
            labels = self.transform(labels)

        return (features, labels)

## Dataset & Dataloader

In [6]:
# Transform function
def toTensor(x):
  return torch.tensor(x)

In [7]:
# dataset
train_dataset = StockDataset(
    file_path=r'Stock.xlsx',
    seq_length=5,
    train=True,
    transform=toTensor
    )

test_dataset = StockDataset(
    file_path = 'Stock.xlsx',
    seq_length = 5,
    train = False,
    transform = toTensor
)

In [8]:
fetures, labels = train_dataset[0:7]
fetures.shape, labels.shape

(torch.Size([7, 5, 4]), torch.Size([7, 5, 1]))

In [9]:
# dataloader
train_loader = DataLoader(
    dataset = train_dataset, 
    batch_size = 4,
    shuffle = True
    )

test_loader = DataLoader(
    dataset = test_dataset, 
    batch_size = 4,
    shuffle = False
)

In [10]:
for idx, (seqs, labels) in enumerate(train_loader):
    print(seqs.shape)
    print(seqs)
    break

torch.Size([4, 5, 4])
tensor([[[3272.1900, 3286.6700, 3265.7600, 3281.0000],
         [3280.6200, 3295.1900, 3275.0500, 3286.6200],
         [3285.4600, 3285.4600, 3265.0100, 3269.3900],
         [3266.2200, 3290.3900, 3244.4000, 3288.9700],
         [3283.8400, 3284.9300, 3262.2800, 3273.8300]],

        [[3148.0200, 3150.4600, 3134.6100, 3140.0100],
         [3148.9900, 3157.0300, 3132.6200, 3156.2100],
         [3152.2400, 3186.9800, 3146.6400, 3147.4500],
         [3138.4400, 3158.0500, 3118.0900, 3157.8700],
         [3157.0000, 3187.8900, 3156.9800, 3185.4400]],

        [[3242.2200, 3254.3400, 3239.8800, 3253.3300],
         [3252.6900, 3261.3800, 3243.8400, 3261.2200],
         [3258.8300, 3264.0800, 3236.3500, 3251.3800],
         [3246.8600, 3253.9600, 3233.5300, 3253.4300],
         [3249.1900, 3251.6500, 3224.0900, 3228.6600]],

        [[3054.1100, 3090.4900, 3051.8700, 3083.5100],
         [3085.9300, 3098.9100, 3085.9300, 3090.2300],
         [3082.8700, 3113.5100, 3060.

## LSTM Model

**Note:**
* `batch_first` – If `True`, then the input and output tensors are provided as `(batch, seq, feature)` instead of `(seq, batch, feature)`. 

- from [LSTM Documentation](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

In [11]:
# hyper-parameters
input_size = 4
hidden_size = 32 # memory_cell
num_layers = 1
num_class = 1 # output_layer

In [13]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_class):
    super(LSTM, self).__init__()
    
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    
    self.normal = nn.BatchNorm1d(5, affine=True) 
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) # batch_first batch 為第一個維度
    self.fc = nn.Linear(hidden_size, num_class)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, x):
    # layer num, batch size, hidden size

    h0 = torch.zeros(num_layers, x.size(0), hidden_size)    
    c0 = torch.zeros(num_layers, x.size(0), hidden_size)
    out = self.normal(x)

    #lstm output will be output and c_n's hidden state, cell state
    # in pytorch output is a list of each cell's hidden state
    out, (h_out, c) = self.lstm(out, (h0, c0))

    out = self.fc(out)
    out = self.sigmoid(out)

    return out

In [14]:
model = LSTM(input_size, hidden_size, num_layers, num_class)
model

LSTM(
  (normal): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(4, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [16]:
# fed stock data to our model 
for idx, (seqs, labels) in enumerate(train_loader):
    input = seqs.float()
    output = model(input)
    print('Input Data Shape:', seqs.shape)
    print('Output Shape:', output.size())
    break

Input Data Shape: torch.Size([4, 5, 4])
Output Shape: torch.Size([4, 5, 1])


## Loss Function & Optimizer
* Binary Cross Entropy Loss

In [17]:
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

## Training Model

In [18]:
for epoch in range(2000):
  for idx, (seqs, labels) in enumerate(train_loader):
    seqs = seqs.float()
    labels = labels.float()
    
    optimizer.zero_grad()
    outputs = model(seqs)
    loss = loss_func(outputs, labels) # crossentropy need float number
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
      print('Epoch: %d, batch: %d, Loss: %.4f'%(epoch+1, idx+1, loss.data))

Epoch: 100, batch: 1, Loss: 0.6648
Epoch: 100, batch: 2, Loss: 0.6972
Epoch: 100, batch: 3, Loss: 0.6740
Epoch: 100, batch: 4, Loss: 0.6549
Epoch: 100, batch: 5, Loss: 0.6607
Epoch: 100, batch: 6, Loss: 0.6804
Epoch: 100, batch: 7, Loss: 0.7018
Epoch: 200, batch: 1, Loss: 0.7036
Epoch: 200, batch: 2, Loss: 0.6180
Epoch: 200, batch: 3, Loss: 0.5833
Epoch: 200, batch: 4, Loss: 0.7060
Epoch: 200, batch: 5, Loss: 0.6684
Epoch: 200, batch: 6, Loss: 0.7125
Epoch: 200, batch: 7, Loss: 0.6062
Epoch: 300, batch: 1, Loss: 0.6293
Epoch: 300, batch: 2, Loss: 0.6997
Epoch: 300, batch: 3, Loss: 0.7605
Epoch: 300, batch: 4, Loss: 0.6984
Epoch: 300, batch: 5, Loss: 0.6318
Epoch: 300, batch: 6, Loss: 0.6493
Epoch: 300, batch: 7, Loss: 0.6824
Epoch: 400, batch: 1, Loss: 0.6161
Epoch: 400, batch: 2, Loss: 0.6572
Epoch: 400, batch: 3, Loss: 0.7039
Epoch: 400, batch: 4, Loss: 0.6604
Epoch: 400, batch: 5, Loss: 0.6995
Epoch: 400, batch: 6, Loss: 0.5955
Epoch: 400, batch: 7, Loss: 0.7982
Epoch: 500, batch: 1

## Testing Model

In [19]:
correct = 0
total = 0
for seqs, labels in test_loader:
  seqs = seqs.float()
  outputs = model(seqs).float()

  predicted = outputs.gt(0.5) # bool
  total += (labels.size(0) * labels.size(1))
  correct += (predicted == labels).sum()

print('Acc: %.3f %%' % (100.0 * float(correct)//float(total)))

Acc: 61.000 %
